# 00_baseline Playground

이 노트북은 **Pure Naive RAG (V0)** 파이프라인을 실험하는 공간입니다.
메타데이터 필터링 없이, 단순한 Vector Search 기반의 검색 및 답변 생성을 테스트합니다.

**사전 준비**:
1. `.env` 파일에 HyperCLOVA X 및 Milvus 설정 완료
2. `python 00_baseline/ingest.py` 실행하여 데이터 적재 완료

In [1]:
# Auto-reload ensures that if you edit pipeline.py, changes are reflected here immediately
%load_ext autoreload
%autoreload 2

import sys
import os
from dotenv import load_dotenv

load_dotenv()

# Add root directory to sys.path to import config
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# Import the pipeline class from our structured file
from pipeline import RAGPipelineV0
from config import Config

## 1. Milvus 적재 상태 확인
데이터가 잘 들어갔는지 확인해봅니다.

In [2]:
from pymilvus import MilvusClient

client = MilvusClient(uri=Config.MILVUS_URI, token=Config.MILVUS_TOKEN)
collection_name = Config.MILVUS_COLLECTION_NAME_V0

if client.has_collection(collection_name):
    count = client.query(collection_name=collection_name, filter="", output_fields=["count(*)"])[0]["count(*)"]
    print(f"Collection '{collection_name}' exists with {count} entities.")
else:
    print(f"Collection '{collection_name}' does not exist. Please run ingest.py first.")

Collection 'audit_reports_v0' exists with 13064 entities.


## 2. 파이프라인 초기화 및 테스트

In [3]:
pipeline = RAGPipelineV0()
print("Pipeline Initialized!")

Pipeline Initialized!


### 2.1. 단순 검색 테스트 (Retrieval Only)
질문에 대해 어떤 문서들이 검색되는지 확인합니다.

In [4]:
query = "한국철도공사의 예산 관련 지적사항은?"
retrieved_docs = pipeline.retriever.invoke(query)

for i, doc in enumerate(retrieved_docs):
    print(f"[{i+1}] {doc.metadata.get('title', 'No Title')} (Score: N/A in LangChain retriever)")
    print(f"Preview: {doc.page_content[:100]}...\n")

[1] 단순노무 용역 퇴직급여충당금 정산 부적정 (Score: N/A in LangChain retriever)
Preview: Contents: 시 정 요 구 제 목 단순노무 용역 퇴직급여충당금 정산 부적정 소 관 부 서 △△△△△ ㈜ 조 치 부 서 □□□□ 본부 ▽▽▽ 처 등 x개 소속 내 용 1. 업무...

[2] 위탁사업비 정산자료 누락에 따른 공단 정산 미지급 우려 (Score: N/A in LangChain retriever)
Preview: Problem: 감사결과 확인된 문제 그런데도 위 부서는 20xx.xx.xx. CCDD단에 위탁사업비 정산자료를 제출하면서 20xx년도 x월에 시행한 CC 및 DD분야 교육 건에 ...

[3] 시설물 내진성능평가 용역 입찰에서 신기술 활용실적 점수 산정 부정확으로 낙찰자 결정 및 계약 체결 (Score: N/A in LangChain retriever)
Preview: Contents: 감 사 원 주 의 요 구 제 목 시설물 내진성능평가 용역 계약업무 처리 부적정 소 관 기 관 국가철도공단 조 치 기 관 국가철도공단 내 용 1. 업무 개요 국가철...

[4] 광역철도 위탁사업비 정산 미비 및 광역교통개선대책 사업비 집행 관리 미흡 (Score: N/A in LangChain retriever)
Preview: Problem: 1. 광역철도 위탁사업비 정산관리 미흡: 2021년 12월 현재까지 준공된 6개 광역철도 사업의 최종 사업비 정산보고서를 제출하지 않아 분담금 잔액이 방치되었음, ...

[5] 장기대여학자금 회수금 수입처리 미반영 및 예산총계주의 위반 (Score: N/A in LangChain retriever)
Preview: Contents: 감 사 원 주 의 요 구 제 목 장기대여학자금 회수금 수입예산 처리 부적정 소 관 기 관 국가철도공단 조 치 기 관 국가철도공단 내 용 1. 업무 개요 국가철도공...



### 2.2. 전체 답변 생성 테스트 (Generation)
실제 RAG 답변을 생성해봅니다.

In [5]:
answer = pipeline.run(query)
print("\n=== Answer ===\n")
print(answer)

Processing query (V0): 한국철도공사의 예산 관련 지적사항은?

=== Answer ===

한국철도공사와 관련된 예산 관련 지적 사항은 다음과 같습니다.

1. **광역철도 위탁사업비 정산 관리 미흡**: 2021년 12월 현재까지 준공된 6개 광역철도 사업의 최종 사업비 정산 보고서를 제출하지 않아 분담금 잔액이 방치되었습니다. 잔액은 총 22,600백만 원이며, 이 중 국비는 3,015백만 원, 지방비는 19,585백만 원이 포함되어 있습니다.

2. **광역교통 개선 대책 사업비 집행 관리 미흡**: LH가 실제 집행비를 분리 관리하지 못하고 추정 사업비를 바탕으로 이행 상황을 보고하여 차액이 누적되었으며, 7개 세부사업에서 추정 사업비와 실제 집행 금액 차이가 30% 이상 발생했습니다. 이로 인해 새로운 교통시설 확충에 차액이 제대로 활용되지 못할 우려가 있습니다.
